# Extreme learning Machine


In [40]:
import tensorflow as tf
import numpy as np
X = np.array([
    [0.85, 0.32, 0.68],  # Gabor feature 1, Gabor feature 2, Gabor feature 3 (Tumor)
    [0.23, 0.234, 0.1]])
y= np.array([10, 90])  # 1 for Tumor, 0 for Healthy


class ELM:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Randomly initialize input weights and biases (key ELM aspect)
        self.input_weights = tf.Variable(tf.random.normal([input_size, hidden_size]))
        self.biases = tf.Variable(tf.random.normal([hidden_size]))

        # Output weights will be learned
        self.output_weights = tf.Variable(tf.zeros([hidden_size, output_size]))

    def hidden_layer(self, X):
        X = tf.cast(X, tf.float32)  # Cast X to float32
        H = tf.nn.sigmoid(tf.matmul(X, self.input_weights) + self.biases)  # Activation function
        return H

    def train(self, X, y):
        H = self.hidden_layer(X)
        # Solve for output weights using pseudo-inverse (or regularized methods)
        y = tf.cast(y, tf.float32) # Cast y to float32
        self.output_weights.assign(tf.linalg.pinv(H) @ tf.reshape(y, [-1, 1])) # Reshape y to a column vector

    def predict(self, X):
        H = self.hidden_layer(X)
        y_pred = tf.matmul(H, self.output_weights)
        return y_pred

# Example Usage
model = ELM(input_size=X.shape[1], hidden_size=6, output_size=1)  # 24 hidden nodes
model.train(X, y)
predictions = model.predict(X)
print("--"*26)
print(predictions)
print("--"*26)

----------------------------------------------------
tf.Tensor(
[[10.000015]
 [90.00001 ]], shape=(2, 1), dtype=float32)
----------------------------------------------------


Random Initialization: The input_weights and biases are randomly initialized and not updated during training. This is a fundamental characteristic of ELMs that contributes to their speed.

Activation Function: The code uses the sigmoid activation function (tf.nn.sigmoid), but you can experiment with other functions like ReLU, tanh, etc.

Output Weight Calculation: The output weights are calculated using the pseudo-inverse (tf.linalg.pinv).

Hidden Layer Size: The paper uses 24 hidden nodes (hidden_size=24). You might need to tune this hyperparameter
based on your dataset and problem complexity.
TensorFlow Integration: This implementation leverages TensorFlow for efficient matrix operations and potential GPU acceleration.

# ELMRegressor_TensorFlow

In [46]:

class ELMRegressor_TF:
    def __init__(self, n_hidden_units):
        self.n_hidden_units = n_hidden_units

    def fit(self, X, labels):
        # Convert inputs to tensors
        X = tf.constant(X, dtype=tf.float32)
        labels = tf.constant(labels, dtype=tf.float32)

        # Add bias column (ones)
        X = tf.concat([X, tf.ones([tf.shape(X)[0], 1], dtype=tf.float32)], axis=1)

        # Initialize random weights and calculate G (hidden layer output)
        self.random_weights = tf.Variable(
            tf.random.normal([tf.shape(X)[1], self.n_hidden_units], dtype=tf.float32)
        )
        G = tf.nn.tanh(tf.matmul(X, self.random_weights))

        # Compute output weights (w_elm) using pseudo-inverse
        self.w_elm = tf.matmul(tf.linalg.pinv(G), labels)

    def predict(self, X):
        # Convert input and add bias
        X = tf.constant(X, dtype=tf.float32)
        X = tf.concat([X, tf.ones([tf.shape(X)[0], 1], dtype=tf.float32)], axis=1)

        # Calculate hidden layer output and predictions--> REGRESSOR
        G = tf.nn.tanh(tf.matmul(X, self.random_weights))
        return tf.matmul(G, self.w_elm)


# difference b/w both ELM model first and 2nd implementation?

# For most regression tasks, ELMRegressor_TF (using tanh activation and implicit bias handling)
#is generally preferred or considered a more robust choice for the hidden layer activation function.

In [48]:
# Sample data
X = tf.constant([[1, 2], [3, 4], [5, 6]], dtype=tf.float32)
labels = tf.constant([[10], [20], [30]], dtype=tf.float32)

# Create and train the model
elm_regressor = ELMRegressor_TF(n_hidden_units=10)
elm_regressor.fit(X, labels)

# Make predictions
new_X = tf.constant([[7, 8], [9, 10]], dtype=tf.float32)
predictions = elm_regressor.predict(new_X)
print(predictions)


tf.Tensor(
[[39.679764]
 [49.39465 ]], shape=(2, 1), dtype=float32)


# eXTREME Machine Learning (ELM) Regressor

perform better than normal

In [49]:
#now normal

class ELM_TF:
    def __init__(self, input_size, output_size, hidden_size):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size

        # TensorFlow Variable initialization for weights and bias
        self.weight = tf.Variable(
            tf.random.normal([self.hidden_size, self.input_size], -0.5, 0.5, dtype=tf.float32)
        )
        self.bias = tf.Variable(
            tf.random.normal([1, self.hidden_size], 0, 1, dtype=tf.float32)
        )
        self.beta = None  # Will be initialized during training

    def sigmoid(self, x):
        return tf.math.sigmoid(x)  # TensorFlow's sigmoid activation

    def predict(self, X):
        X = tf.cast(X, tf.float32)
        H = self.sigmoid(tf.matmul(X, tf.transpose(self.weight)) + self.bias)
        return tf.matmul(H, self.beta)

    def train(self, X, y):
        X = tf.cast(X, tf.float32)
        y = tf.cast(y, tf.float32)

        # Calculate hidden layer output
        H = self.sigmoid(tf.matmul(X, tf.transpose(self.weight)) + self.bias)

        # Compute beta (output weights) using TensorFlow's pseudo-inverse
        H_pinv = tf.linalg.pinv(tf.matmul(tf.transpose(H), H))
        self.beta = tf.matmul(tf.matmul(H_pinv, tf.transpose(H)), y)

        # Return training predictions (optional, for evaluation)
        return tf.matmul(H, self.beta)


In [53]:
# Sample training data
X_train = tf.constant([[1, 2], [3, 4], [5, 6], [7,8]], dtype=tf.float32)
y_train = tf.constant([[10], [20], [30], [40]], dtype=tf.float32)

# Create and train the model
elm_tf = ELM_TF(input_size=2, output_size=1, hidden_size=10)
elm_tf.train(X_train, y_train)

# Make predictions on new data
X_test = tf.constant([[7, 8], [9, 10]], dtype=tf.float32)
predictions = elm_tf.predict(X_test)
print(predictions)


tf.Tensor(
[[40.      ]
 [48.164974]], shape=(2, 1), dtype=float32)


### RESULTS ARE little bit AWESOME BUT VERY CLOSE, BECZ THE TRAINING DATASET IN VERY SMALL

# Classification

In [65]:

class ELM_classifier:
    def __init__(self, input_size, output_size, hidden_size):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size

        # TensorFlow Variable initialization for weights and bias
        self.weight = tf.Variable(
            tf.random.uniform([self.hidden_size, self.input_size], -1, 1, dtype=tf.float32)
        )
        self.bias = tf.Variable(
            tf.random.uniform([self.hidden_size], -1, 1, dtype=tf.float32)
        )
        self.beta = None  # Will be initialized during training

    def sigmoid(self, x):
        return tf.math.sigmoid(x)  # TensorFlow's sigmoid activation

    def predict(self, X):
        X = tf.cast(X, tf.float32)
        H = self.sigmoid(tf.matmul(X, tf.transpose(self.weight)) + self.bias)
        return tf.matmul(H, self.beta)

    def train(self, X, y):
        X = tf.cast(X, tf.float32)
        y = tf.cast(y, tf.float32)
        y = tf.reshape(y, [-1, 1])  # Ensure y is a column vector

        # Calculate hidden layer output
        H = self.sigmoid(tf.matmul(X, tf.transpose(self.weight)) + self.bias)

        # Compute beta (output weights) using TensorFlow's pseudo-inverse or other methods
        # Here's the pseudo-inverse approach:
        H_pinv = tf.linalg.pinv(H)
        self.beta = tf.matmul(H_pinv, y)

        # Return training predictions (optional, for evaluation)
        return tf.matmul(H, self.beta)

# Example usage (same as before)
X_train = np.array([
    [0.85, 0.32, 0.68],
    [0.12, 0.91, 0.25],
    [0.79, 0.45, 0.55],
    [0.21, 0.88, 0.33],
    [0.92, 0.28, 0.71]
])

y_train = np.array([1, 0, 1, 0, 1])

X_train = tf.constant(X_train, dtype=tf.float32)
y_train = tf.constant(y_train, dtype=tf.float32)

# Create and train the model
elm_tf_classifier = ELM_classifier(input_size=3, output_size=1, hidden_size=10) # we have three columns so input size =3
elm_tf_classifier.train(X_train, y_train)


<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[ 1.0000005e+00],
       [-2.8610229e-06],
       [ 9.9999285e-01],
       [ 0.0000000e+00],
       [ 1.0000014e+00]], dtype=float32)>

# train data

In [66]:

guess = elm_tf_classifier.predict(X_train)
predictions = guess.numpy()  # Convert tensor to NumPy array
print(predictions)
for i, prediction in enumerate(predictions):
    if prediction > 0.5:
        print(f"Sample {i+1}: Tumor (Probability: {prediction[0]:.2f})")
    else:
        print(f"Sample {i+1}: Healthy (Probability: {1 - prediction[0]:.2f})")


[[ 1.0000005e+00]
 [-2.8610229e-06]
 [ 9.9999285e-01]
 [ 0.0000000e+00]
 [ 1.0000014e+00]]
Sample 1: Tumor (Probability: 1.00)
Sample 2: Healthy (Probability: 1.00)
Sample 3: Tumor (Probability: 1.00)
Sample 4: Healthy (Probability: 1.00)
Sample 5: Tumor (Probability: 1.00)


# test data

In [67]:
X_test = np.array([
    [0.81, 0.38, 0.62],  # Gabor feature 1, Gabor feature 2, Gabor feature 3 (Tumor)
    [0.18, 0.85, 0.30]   # ... (Healthy)
])

y_test = np.array([1, 0])  # 1 for Tumor, 0 for Healthy


X_test = tf.constant(X_test, dtype=tf.float32)
y_test = tf.constant(y_test, dtype=tf.float32)

guess_test = elm_tf_classifier.predict(X_test)
guess_test = guess_test.numpy()  # Convert tensor to NumPy array
print(guess_test)

for i, prediction_1 in enumerate(guess_test):
    if prediction_1 > 0.5:
        print(f"Sample {i+1}: Tumor (Probability: {prediction[0]:.2f})")

    else:
        print(f"Sample {i+1}: Healthy (Probability: {1 - prediction[0]:.2f})")

[[1.0056949 ]
 [0.14818382]]
Sample 1: Tumor (Probability: 1.00)
Sample 2: Healthy (Probability: -0.00)
